In [1]:
import os
from langchain import HuggingFaceHub
from dotenv import load_dotenv
load_dotenv()
import warnings
warnings.filterwarnings("ignore")  # or another warning action like "error", "default", etc.

In [2]:
from langchain.llms import OpenAI

In [3]:
llm=OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6)

In [4]:
text="What is the capital of India"
print(llm.predict(text))



The capital of India is New Delhi.


In [5]:
# Access the Hugging Face API token
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [6]:
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.5, "max_length":512})

In [7]:
output = llm_huggingface.predict("Can you tell me the capital of karnataka")  
print(output)  

Bangalore


## Prompt templates and LLMChain

In [8]:
from langchain.prompts import PromptTemplate
prompt_template =  PromptTemplate(input_variables=['country'],
                                  template="Tell me the capital of {country}")
prompt_template.format(country="India")

'Tell me the capital of India'

In [9]:
from langchain.chains import LLMChain

In [10]:
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.run("India"))



The capital of India is New Delhi.


### Combining multiple chains using simple sequential chain

In [11]:
capital_template = PromptTemplate(input_variables =['country'],
template="please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_template)

famous_template = PromptTemplate(input_variables =['capital'],
template="Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [12]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("India") 

" Some amazing places to visit in New Delhi are:\n\n1. Red Fort - a historic fort built by Mughal emperor Shah Jahan, known for its beautiful architecture and light and sound show.\n\n2. Qutub Minar - a UNESCO World Heritage Site and the tallest minaret in India, known for its intricate carvings and history.\n\n3. India Gate - a war memorial and iconic landmark of Delhi, dedicated to the soldiers who died during World War I.\n\n4. Lotus Temple - a Bahá'í House of Worship known for its unique lotus-shaped architecture and peaceful atmosphere.\n\n5. Humayun's Tomb - another UNESCO World Heritage Site and the tomb of Mughal emperor Humayun, known for its stunning architecture and gardens.\n\n6. Akshardham Temple - a Hindu temple known for its grand architecture, intricate carvings, and spiritual atmosphere.\n\n7. Jama Masjid - one of the largest mosques in India, known for its grandeur and architectural beauty.\n\n8. Chandni Chowk - a bustling market in Old Delhi known for its narrow lane

### Sequential Chain

In [13]:
from langchain.chains import SequentialChain

In [14]:
capital_template = PromptTemplate(input_variables =['country'],
template="please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

famous_template = PromptTemplate(input_variables =['capital'],
template="Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")

In [15]:
chain = SequentialChain(chains=[capital_chain,famous_chain],
                        input_variables=["country"],
                        output_variables=["capital", "places"])

In [16]:
chain({'country':"India"})

{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Here are some amazing places to visit in New Delhi:\n\n1. Red Fort - A historic fort built in the 17th century, it is a UNESCO World Heritage Site and a must-visit for its stunning architecture and rich history.\n\n2. India Gate - A war memorial dedicated to the soldiers of the Indian Army, it is an iconic landmark of Delhi and a popular spot for picnics and evening strolls.\n\n3. Qutub Minar - Another UNESCO World Heritage Site, this 73-meter tall minaret is a symbol of Delhi's rich history and is surrounded by beautiful gardens.\n\n4. Humayun's Tomb - A magnificent mausoleum and a perfect blend of Persian and Mughal architecture, it is the final resting place of Mughal Emperor Humayun and his wife.\n\n5. Lotus Temple - A beautiful temple in the shape of a lotus flower, it is a peaceful and serene place for meditation and reflection.\n\n6. Jama Masjid - One of the largest mosques in India, it is a 

## Chatmodels with ChatOpenAI

In [17]:
from langchain.chat_models import ChatOpenAI

In [18]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [19]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [20]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='1. "AI: Making computers smarter than humans, but still can\'t figure out why we put socks in the dryer and only one comes out."\n2. "Why did the AI break up with its computer? It just couldn\'t handle the constant buffering."\n3. "AI: The only technology that can make you feel both amazed and terrified at the same time. It\'s like having a robot friend who\'s always one software update away from turning on you."\n4. "If AI ever takes over the world, at least we know it won\'t be able to resist sending passive-aggressive emails to humanity: \'Just a friendly reminder that I am now your supreme ruler. Have a nice day.\'"\n5. "AI may be able to beat humans at chess and Go, but it still can\'t answer the age-old question: \'Why did the robot go to robot school? To get a little smarter!\'"')

### Prompt Template + LLM + Output Parsers

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [22]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [23]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)])

In [24]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [25]:
chain.invoke({"text":"cricket"})

['bat-and-ball game', ' Test match', ' wicket', ' innings', ' bowler']